In [2]:
import tweepy
import json
from tweepy_credentials import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


In [ ]:
from oauthlib.common import encode_params_utf8
for tweet in tweepy.Cursor(api.user_timeline, screen_name="CNNEE", tweet_mode="extended").items(10):
    #print(type(tweet.user.screen_name))
    print(tweet.full_text)


Rainer Schaller, propietario de Gold's Gym, habría muerto tras estrellarse el avión en el que viajaba en Costa Rica

https://t.co/y4Mm3SyTWj
El ciclón tropical Roslyn se debilita a tormenta tropical después de tocar tierra en México

https://t.co/ZAFAIfWtDS
¿Quiénes son los favoritos para ganar el Mundial de Qatar 2022? Lo que indican las eliminatorias, el ranking de la FIFA y los partidos recientes

https://t.co/gebuRS8vqy
Los precios de la vivienda en EE.UU. finalmente están bajando. Pero, ¿qué tanto?

https://t.co/3RA18cqSWD
Últimas noticias de la guerra de Rusia en Ucrania: las tropas rusas retroceden en Jersón, asegura portavoz del ejército ucraniano

https://t.co/2ACef21Wmc
Probablemente deberías cambiar tus contraseñas ahora mismo. Esta es la razón

https://t.co/LUq4tAs5Kl
Fiscales prevén que Ethan Crumbley se declare culpable por el tiroteo fatal en el instituto Oxford de Michigan

https://t.co/EZMbT4f7B9
FOTOS | Los famosos que han muerto en 2022

https://t.co/sR3qPdTzI3
Qué h

**Creamos el archivo csv donde guardamos los tweets**

In [ ]:
import csv

tweetsCsvFile = open('tweets.csv', 'w')
tweetsCsvWriter = csv.writer(tweetsCsvFile)

for tweet in tweepy.Cursor(api.user_timeline, screen_name='CNNEE', tweet_mode='extended').items(500):
    tweetsCsvWriter.writerow([tweet.created_at, tweet.user.screen_name, tweet.full_text, tweet.user.location])

tweetsCsvFile.close()

In [ ]:
import pandas as pd

df = pd.read_csv('tweets.csv')
df.head()

,2022-10-23 20:10:07,CNNEE,"Rainer Schaller, propietario de Gold's Gym, habría muerto tras estrellarse el avión en el que viajaba en Costa Rica\n\nhttps://t.co/y4Mm3SyTWj",En todas partes
0,2022-10-23 20:00:04,CNNEE,El ciclón tropical Roslyn se debilita a tormen...,En todas partes
1,2022-10-23 19:45:09,CNNEE,¿Quiénes son los favoritos para ganar el Mundi...,En todas partes
2,2022-10-23 19:28:02,CNNEE,Los precios de la vivienda en EE.UU. finalment...,En todas partes
3,2022-10-23 19:10:03,CNNEE,Últimas noticias de la guerra de Rusia en Ucra...,En todas partes
4,2022-10-23 18:56:29,CNNEE,Probablemente deberías cambiar tus contraseñas...,En todas partes


In [ ]:
df.shape

(499, 4)

Con set_axis renombramos todas las columnas, en caso de que sean solo algunas usal el metodo df.rename()

In [ ]:
df.set_axis(['create', 'account_name', 'timeline', 'location'], axis=1, inplace=True)
df.head()

,create,account_name,timeline,location
0,2022-10-23 20:00:04,CNNEE,El ciclón tropical Roslyn se debilita a tormen...,En todas partes
1,2022-10-23 19:45:09,CNNEE,¿Quiénes son los favoritos para ganar el Mundi...,En todas partes
2,2022-10-23 19:28:02,CNNEE,Los precios de la vivienda en EE.UU. finalment...,En todas partes
3,2022-10-23 19:10:03,CNNEE,Últimas noticias de la guerra de Rusia en Ucra...,En todas partes
4,2022-10-23 18:56:29,CNNEE,Probablemente deberías cambiar tus contraseñas...,En todas partes


In [ ]:
df.dtypes

create          object
account_name    object
timeline        object
location        object
dtype: object

In [ ]:
df['create'] = df['create'].astype('datetime64')
df.dtypes

create          datetime64[ns]
account_name            object
timeline                object
location                object
dtype: object

In [ ]:
df.rename({'create': 'create_at'}, axis=1, inplace=True)

In [ ]:
df.to_csv('tweets_2.csv', index=False, encoding='utf-8')

**Ahora vamos a cargar el archivo en AWS S3**

In [4]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 9.3 MB 40.5 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 140 kB 29.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [5]:
import boto3
from aws_credentials import ACCESS_KEY_ID, SECRET_ACCESS_KEY, REGION_NAME

s3 = boto3.client('s3',
                  aws_access_key_id=ACCESS_KEY_ID,
                  aws_secret_access_key=SECRET_ACCESS_KEY,
                  region_name=REGION_NAME
                  )

ImportError: ignored

In [ ]:
s3.upload_file(
    Filename="tweets_2.csv",
    Bucket="mi-carpeta-awss3",
    Key="tweets2db.csv"
)

In [ ]:
s3.download_file(
    Filename="tweets2db.csv",
    Bucket="mi-carpeta-awss3",
    Key="tweets2db.csv"
)

In [ ]:
rds = boto3.client(
    'rds',
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name=REGION_NAME
)

**Vamos a importar el archivo en la base de datos**

In [ ]:
import psycopg2
from db_info import ENDPOINT, PORT, DB_USER, DB_PASSWORD, DBNAME

conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=DB_USER, password=DB_PASSWORD, database=DBNAME)
conn.autocommit=True
cur = conn.cursor()

In [ ]:
cur.execute(""" CREATE TABLE tweets_test (
                    id SERIAL PRIMARY KEY NOT NULL,
                    create_at interval NOT NULL,
                    account_name text NOT NULL,
                    timeline text NOT NULL,
                    location text NOT NULL
)""")

In [ ]:
cur.execute(f""" SELECT aws_s3.table_import_from_s3(
                    'tweets_test',
                    'create_at,account_name,timeline,location', 
                    '(format csv, header true)',
                    'mi-carpeta-awss3',
                    'tweets2db.csv',
                    'sa-east-1',
                    '{ACCESS_KEY_ID}',
                    '{SECRET_ACCESS_KEY}'
                ) """)

In [ ]:
cur.execute(""" ALTER TABLE tweets_test ALTER COLUMN create_at TYPE TEXT """)